# __Project Description__


Dataset: https://archive.ics.uci.edu/ml/datasets/bank+marketing

The goal of the project is to determine will the client subscribe a bank term deposit

## __Referenced papers__
1. https://ieeexplore.ieee.org/abstract/document/9065648
2. https://www.researchgate.net/publication/323198261_Customer_Profiling_using_Classification_Approach_for_Bank_Telemarketing
3. https://ieeexplore.ieee.org/abstract/document/8391441
4. https://ieeexplore.ieee.org/document/9655824

# __Exploratory Data Analysis__

First we read the dataset into dataframe. Notice that the dataset is a csv file but separated by `;`

In [ ]:
import pandas as pd

df = pd.read_csv("./dataset/bank-additional/bank-additional.csv",sep=";")

df.head()

Drop column __duration__, according to UCI:    

> ` this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.`

In [ ]:
df.drop(columns="duration", inplace=True)


Getting info of dataframe. There is no null data in any of the feature. So we don't need to handle this case

In [ ]:
df.info(verbose=True, show_counts=True)


### __Visualization__

Visualizing distribution of categorical features and label using pie chart. As can be seen, we have a fairly imbalanced data


In [ ]:
import matplotlib.pyplot as plt

pie_visualization = ["job", "marital", "education",
                     "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome",  "y"]

fig, axes = plt.subplots(3, 4, figsize=(18, 10))

for i, feature in enumerate(pie_visualization):
    category = df[feature].value_counts()
    axes.flat[i].set_title("Distribution of {}".format(feature))
    axes.flat[i].pie(category, labels=category.index, autopct='%1.1f%%')


Visualizing when the data is normally collected. As can be seen, most of the data is collected around May. The day_of_week is fairly balanced

In [ ]:
bar_visualization = ["day_of_week", "month"]
fig, axes = plt.subplots(2, 1, figsize=(8, 10))
for i, ax in enumerate(axes.flat, start=0):
    feature = bar_visualization[i]
    df[feature].value_counts().plot(kind="barh", ax=ax,title="Frequency of {}".format(feature))


Visualizing correlation between numeric features    

As can be noticed from the graph, `emp.var.rate`, `nr.employed`, `euribor3m`,`cons.price.index` are highly correlated
Depending on the algorithm, we may need remove one of the above listed features. This is a **Multicollinearity** problem 

In [ ]:
import seaborn as sb
from matplotlib import rcParams

# figure size in inches
rcParams['figure.figsize'] = 8, 5
numerical_features = ["age", "campaign", "pdays", "previous", "emp.var.rate",
                      "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]

corr = df[numerical_features].corr()
print(type(corr))
sb.heatmap(corr, cmap="Blues", annot=True)


For numerical features, we are also interested in their distributions. 

As can be seen, there are lots of outliers in age, campaign, pdays, previous

In [ ]:
from math import ceil


fig, axes = plt.subplots(ceil(len(numerical_features)/2),2, figsize=(10, 10))

for i, feature in enumerate(numerical_features):
    df.boxplot(column=feature, ax=axes.flat[i])


For features with many outliers, count frequency

We notice that, people between 25 and 60 years old will likely to invest.

For `campaign`, there are many who gets contacted by bank representative by the first time, the same can be said for `pdays` and `previous`.

Potentially, we need to remove outliers from the 4 mentioned features

In [ ]:
feature_outliers = ["campaign", "pdays", "previous"]
fig, axes = plt.subplots(2, 2, figsize=(8, 10))

df["age"].value_counts(bins=10).plot(kind="barh", title="Frequency of age")

for i, feature in enumerate(feature_outliers):
    df[feature].value_counts().plot(
        kind="barh", ax=axes.flat[i], title="Frequency of {}".format(feature))


### __Prepare data for ML algorithms__

Some ML algorithms do not work well with categorical data, therefore, we need to convert those categorical data to number.

There are several features that are strictly __nominal__, perform **One Hot Encoder** on these features. Some features are __ordinal__, with these features, we perform **Ordinal Encoder**.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Ordering array from low to high
# The first element has lowest priority, the last element has highest priority
# This ranking is important!!
education = ["illiterate", "basic.4y", "basic.6y", "basic.9y",
             "unknown", "high.school", "professional.course", "university.degree"]

credit_default = ["yes", "unknown", "no"]

poutcome = ["failure", "nonexistent", "success"]

# 0 means no, 1 means yes
y = ["no", "yes"]

columnTransformer = ColumnTransformer(
    [
        ("OneHotEncoder", OneHotEncoder(), [
         "job", "marital", "contact", "housing", "loan", "day_of_week", "month"]),
        ("OrdinalEncoder", OrdinalEncoder(
            categories=[education, credit_default, poutcome, y]),
            ["education", "default", "poutcome", "y"]),
    ],
    remainder="passthrough"
)

transformed_df = pd.DataFrame(
    columnTransformer.fit_transform(df),
    columns=columnTransformer.get_feature_names_out()
)

# Need to find out a way to feature engineering day and month
# Cannot use label encoding these data are cyclical data in nature
# Refer https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html
# Also, should housing and loan be in encoded using OrdinalEncoder or OneHotEncoder?
# I feel like there is no direct ranking between them so to play it safe, I use OneHotEncoder for it

transformed_df.info(verbose=False)


After running the encoder, we notice that the `dtypes` of dataframe is `float64` for all features!
Now we separate features and label

In [ ]:
x = transformed_df.drop(columns=["OrdinalEncoder__y"])
y = transformed_df["OrdinalEncoder__y"]

print(x.info(verbose=False))
print(y.info(verbose=False))

# Algorithms

## Paper 2

Link: https://www.researchgate.net/publication/323198261_Customer_Profiling_using_Classification_Approach_for_Bank_Telemarketing

Using C4.5 algorithm to predict if a client will subscribe a term deposit.  

In theory, decision tree should be able to work with categorical data. However, the current implementation of   
decision tree in scikit-learn 1.1 does not handle categorical data. Refer https://scikit-learn.org/1.1/modules/tree.html    

As a result, we need to use OneHotEncoder and Ordinal Encoder on the features. `transformed_df` variable is a dataframe that
has all features encoded.

There are trades off of doing this, notably higher tree depth and the time it takes for the algorithm to finish is longer...

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn import tree

%matplotlib inline


def report_result(y_pred, y_true, clf):
    cm = confusion_matrix(y_true, y_pred, labels=clf.classes_)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=clf.classes_)
    disp.plot(cmap="Blues")
    print(classification_report(y_true, y_pred))


def tree_visualizer(tree_classifier, feature_names):
    fig = plt.figure(figsize=(25, 20))
    _ = tree.plot_tree(tree_classifier,
                       feature_names=feature_names,
                       class_names=['No', "Yes"],
                       filled=True)


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier

# The following features are not used by the paper, therefore, we're not going to use these features as well
tree_dataset_x = x.drop(columns=["remainder__euribor3m", "remainder__nr.employed",
                                 "remainder__cons.price.idx", "remainder__emp.var.rate", "remainder__cons.conf.idx"], inplace=False)

# Seperate into train and test set
x_train, x_test, y_train, y_test = train_test_split(
    tree_dataset_x, y, train_size=0.8, test_size=0.2, random_state=0)

# Create validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(
    x_train, y_train, train_size=0.6, test_size=0.4, random_state=0)


tree_clf = DecisionTreeClassifier(max_depth=4)

tree_clf.fit(X_train, Y_train)
y_pred = tree_clf.predict(x_test)
report_result(y_pred, y_test, tree_clf)
tree_visualizer(tree_clf, X_train.columns)


Hyper parameter tuning with decision tree

We do not need to scale data in order to use decision tree, thus it will be skipped.

The scoring criteria here will be `balanced_accuracy`, this is to account for our imbalanced dataset

In [ ]:
from sklearn.pipeline import Pipeline

dec_tree = DecisionTreeClassifier(random_state=0)
pipe = Pipeline(steps=[
    ('dec_tree', dec_tree)])

parameters = dict(
    dec_tree__class_weight=["balanced"],
    dec_tree__criterion=['gini', 'entropy'],
    dec_tree__max_depth=[2, 4, 6, 8, 10],
    dec_tree__splitter=["best", "random"],
    dec_tree__max_features=["sqrt", "log2", None]
)

tree_tuning = GridSearchCV(pipe, parameters, scoring=[
                           "accuracy", "recall", "balanced_accuracy"], refit="accuracy")
tree_tuning.fit(X_valid, Y_valid)
print("Best params", tree_tuning.best_params_)


Using tuned parameters on the test dataset

In [ ]:
report_result(tree_tuning.predict(x_test), y_test, tree_tuning)
tree_visualizer(tree_tuning.best_estimator_.get_params()['dec_tree'], x_test.columns)

The paper result mentioned that on testing data, they achivieved `90.09%`, `59.06%`, `93.23%` on accuracy score, specificity, and sensitivity respectively. 

As for our result, we achieved `91%` on accuracy score, `67%` on specitivity score and finally `91%` on sensitivity score.

Compared to the paper result, we found out that we are better at overall accuracy of the algorithm as well as correctly identifying term deposit subscriber.

However, there are a lot of room for improvement with the algorithm, for example, the recall rate of term deposit subscriber is only at `8%` only. Meaning, we are missing out on a lot of other potential customers.

## Paper 3

Reserch Paper 3 : https://ieeexplore.ieee.org/abstract/document/9065648




In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA 
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

pca = PCA(n_components=2)
pca.fit(x)
feature_pca = pca.transform(x)
X_trainP, X_testP, y_trainP, y_testP = train_test_split(feature_pca, y, random_state=0)

neigh_PCA = KNeighborsClassifier(n_neighbors=4, algorithm = "brute", weights="uniform")
neigh_PCA.fit(X_trainP, y_trainP)

accuracy_PCA = neigh_PCA.score(X_testP, y_testP)
print("Accuracy: %f" % accuracy_PCA)

y_test_pred = neigh_PCA.predict(X_testP)
print("Precision: %f" % precision_score(y_testP, y_test_pred, average='weighted'))
print("Recall: %f" % recall_score(y_testP, y_test_pred, average='weighted'))
print("F1: %f" % f1_score(y_testP, y_test_pred, average='weighted'))





Accuracy: 0.902913
Precision: 0.877134
Recall: 0.902913
F1: 0.881020


### Results
The Accuracy result in the Research Paper of KNN is 89.80 with CfsSubsetEval dimension reduction. While our Accuracy scrore for KNN with PCA dimension reduction is 90.29. 
Clearly our result is better than that of the Research Paper.It can be duduced that our implementation performed better because we used PCA instead of CfsSubsetEval for dimension reduction. The main area of focus in this research paper is accuracy but when comparing other mertics we get the following Results: <br> Our Precision scrore is 0.877 while research paper's is 0.887. <br>Our Recall scrore is 0.9029 while research paper's is 0.898.<br> Our F1 scrore is 0.881 while research paper's is 0.891
